In [22]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 4000

In [44]:
## import dataset into dataframe

df = pd.read_csv('Airline_Delay_Cause.csv')

In [45]:
df.head()

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2022,8,9E,Endeavor Air Inc.,ABY,"Albany, GA: Southwest Georgia Regional",79.0,12.0,5.34,0.59,...,0.0,3.19,0.0,0.0,909.0,384.0,70.0,250.0,0.0,205.0
1,2022,8,9E,Endeavor Air Inc.,ACK,"Nantucket, MA: Nantucket Memorial",124.0,11.0,4.19,0.00,...,0.0,2.32,1.0,1.0,675.0,176.0,0.0,231.0,0.0,268.0
2,2022,8,9E,Endeavor Air Inc.,AEX,"Alexandria, LA: Alexandria International",62.0,4.0,3.18,0.00,...,0.0,0.31,0.0,0.0,193.0,158.0,0.0,21.0,0.0,14.0
3,2022,8,9E,Endeavor Air Inc.,AGS,"Augusta, GA: Augusta Regional at Bush Field",162.0,15.0,5.36,1.31,...,0.0,3.81,1.0,0.0,1110.0,758.0,49.0,149.0,0.0,154.0
4,2022,8,9E,Endeavor Air Inc.,ALB,"Albany, NY: Albany International",123.0,18.0,3.84,1.31,...,0.0,8.32,10.0,0.0,984.0,172.0,41.0,313.0,0.0,458.0


In [43]:
df.groupby('company_location')[['salary_in_usd']].agg([np.mean, np.std, min, max])
#comb.groupby('Income Group')[['GDP']].agg(np.mean).loc[['High income: OECD', 'High income: nonOECD']]

TypeError: _size_dispatcher() missing 1 required positional argument: 'a'